# Assignment 2: Uninformed Search

## N-Queens
Here, you will explore N-Queens classic puzzles from the perspective of
uninformed search. 
In this section, you will develop a solver for the n-queens problem, where in n
queens are to be placed on an n × n chessboard so that no pair of queens can
attack each other. Recall that in chess, a queen can attack any piece that lies in
the same row, column, or diagonal as itself.

### Note:
You are expected to write code where you see **your code here**.  
Make sure you delete the lines with **pass** and **raise NotImplementedError** or your code may not run correctly.

### Section 1: Calculate the number of possible placements of N-Queens on an n × n board
Rather than performing a search over all possible placements
of queens on the board, it is sufficient to consider only those configurations
for which each row contains exactly one queen.

Implement the function num_placements_all(n), which returns the number of all possible placements of n queens on an n × n board without taking any of the chess-specific constraints between queens into account, and the function num_placements_one_per_row(n) that calculates the number of possible placements of n queens on an n × n board such that each row contains exactly one queen.

Think carefully about why this restriction is valid, and note the extent to
which it reduces the size of the search space. You should assume that all
queens are indistinguishable for the purposes of your calculations.

In [1]:
############################################################
# Section 1: Implement the function num_placements_all and num_placements_one_per_row
############################################################

import math

def num_placements_all(n):
    return math.comb(n * n, n)

# your code here


def num_placements_one_per_row(n):
    return n ** n

# your code here


In [2]:
##########################
### TEST YOUR SOLUTION ###
##########################
# concatenate test
num_placements_all_test = num_placements_all(8)
assert num_placements_all_test == 4426165368, "review the calculation"
num_placements_one_per_row_test = num_placements_one_per_row(8)
assert num_placements_one_per_row_test == 16777216, "review the calculation"
print("test passed!")


test passed!


### Section 2: Valid queen position check
Write a function n_queens_valid(board) that accepts such a list and returns True if no queen can attack another, or False otherwise. Note that the board size need not be included as an additional argument to decide whether a particular list is valid.

With the answer to the previous question in mind, a sensible representation for a board configuration is a list of numbers between 0 and n−1, where the ith number designates the column of the queen in row i for 0 < i ≤ n. A complete configuration is then specified by a list containing n numbers, and a partial configuration is specified by a list containing fewer than n numbers. Write a function n_queens_valid(board) that accepts such a list and returns True if no queen can attack another, or False otherwise. Note that the board size need not be included as an additional argument to decide whether a particular list is valid.

Instead of directly implementing the whole function of n_queens_valid, we could break down the final function into several sub functions:
1) diag_okay(row1,col1,row2,col2) 
This function aims to check whether the queens position satisfy the diagnal constraint
2) cross_okay(row1,col1,row2,col2) 
This function aims to check whether the queens position satisfy the rows and columns constraint
3) move_allowed(row1,col1,row2,col2) 
This function put together the function diag_okay and the function cross_okay you already implemented to make the complete check

#### diag_okay function 

In [3]:
def diag_okay(row1,col1,row2,col2):
    return abs(row1 - row2) != abs(col1 - col2)
# your code here



In [4]:
##########################
### TEST YOUR SOLUTION ###
##########################
# diag_okay test
diag_okay_test = diag_okay(3,0,1,2)
assert diag_okay_test == False
print("test passed!")


test passed!


#### cross_okay function 

In [5]:
def cross_okay(row1,col1,row2,col2):
    return col1 != col2
# your code here



In [6]:
##########################
### TEST YOUR SOLUTION ###
##########################
# cross_okay test
cross_okay_test = cross_okay(1,3,4,3)
assert cross_okay_test == False
print("test passed!")


test passed!


#### move_allowed function 

In [7]:
def move_allowed(row1,col1,row2,col2):
    return diag_okay(row1, col1, row2, col2) and cross_okay(row1, col1, row2, col2)
# your code here



In [8]:
##########################
### TEST YOUR SOLUTION ###
##########################
# move_allowed test
move_allowed_test1 = move_allowed(1,4,2,5)
move_allowed_test2 = move_allowed(1,3,2,7)

assert move_allowed_test1 == False, "move_allowed_test1 should be False"
assert move_allowed_test2 == True, "move_allowed_test2 should be True"

print("test passed!")


test passed!


With the help of the function move_allowed, you could work on implementing the function n_queens_valid right now!

In [9]:
############################################################
# Section 2: n_queens_valid
############################################################

def n_queens_valid(board):        
    n = len(board)
    for i in range(n):
        for j in range(i + 1, n):
            if not move_allowed(i, board[i], j, board[j]):
                return False
    return True
# your code here



In [10]:
##########################
### TEST YOUR SOLUTION ###
##########################
# n_queens_valid test
n_queens_valid_test1 = n_queens_valid([0, 0])
n_queens_valid_test2 = n_queens_valid([0, 2])
n_queens_valid_test3 = n_queens_valid([0, 1])
n_queens_valid_test4 = n_queens_valid([0, 3, 1])
assert n_queens_valid_test1 == False, "n_queens_valid_test1 should be False"
assert n_queens_valid_test2 == True, "n_queens_valid_test2 should be True"
assert n_queens_valid_test3 == False, "n_queens_valid_test3 should be False"
assert n_queens_valid_test4 == True, "n_queens_valid_test4 should be True"
print("test passed!")


test passed!


### Section 3: Generate all Valid solutions for N-Queens
Here, write a function n_queens_solutions(n) that returns a list of all valid placements of n queens on an n × n board, using the representation discussed above.

Your solution should be implemented as a depth-first search, where queens are successively placed in empty rows until all rows have been filled. You may find it helpful to define a helper function n_queens_helper(n, board) that yields all valid placements which extend the partial solution denoted by board. And the output may be in any order you see fit.

In n_queens_helper function, the "board" parameter which denotes the queen placements in preious steps; then you could iteratively check every placement_row and placement_col on the previous board and one next step queen placement position utilizing the function move_allowed to generate all valid placements new board in the next one step. 

In [11]:
def n_queens_helper(board, n):
    if len(board) == n:
        # If the board is fully populated with n queens, return this configuration
        return [board]
    
    placement_row = len(board)
    valid_placements = []
    
    for placement_col in range(n):
        if all(move_allowed(placement_row, placement_col, row, col) for row, col in enumerate(board)):
            # If placing a queen at (placement_row, placement_col) is valid, proceed recursively
            new_board = board + [placement_col]
            valid_placements.extend(n_queens_helper(new_board, n))
    
    return valid_placements


In [12]:
##########################
### TEST YOUR SOLUTION ###
##########################
# n_queens_helper test
n_queens_helper_test1 = n_queens_helper([1, 3, 5, 0],7)
n_queens_helper_test2 = n_queens_helper([3, 0, 4],7)
assert n_queens_helper_test1 == [[1, 3, 5, 0, 2], [1, 3, 5, 0, 4]]
assert n_queens_helper_test2 == [[3, 0, 4, 1]]
print("test passed!")


AssertionError: 

The basic idea is to place queens one by one in different rows, starting from the upmost row. When we place a queen in a row, we check for clashes with already placed queens. In the current row, if we find a column for which there is no clash, we add this column as one of the solutions for the current row. If we do not find such a column due to clashes, then we return and find other possible placement.

In [13]:
############################################################
# Section 3: n_queens_solutions
############################################################
import collections

def n_queens_solutions(n):
    return n_queens_helper([], n)
    # Initialize: Visited Set(recommend to use collections.defaultdict(bool));
    #             Frontier
    
    # Method: DFS
    
    
# your code here



In [14]:
##########################
### TEST YOUR SOLUTION ###
##########################
# n_queens_solutions test
n_queens_solutions_test0 = n_queens_solutions(5)
n_queens_solutions_test1 = n_queens_solutions(6)
n_queens_solutions_test2 = len(n_queens_solutions(8))
assert n_queens_solutions_test0 ==[[0, 3, 1, 4, 2],[0, 2, 4, 1, 3],[1, 4, 2, 0, 3],\
 [1, 3, 0, 2, 4],[2, 4, 1, 3, 0],[2, 0, 3, 1, 4],[3, 1, 4, 2, 0],[3, 0, 2, 4, 1],\
 [4, 2, 0, 3, 1],[4, 1, 3, 0, 2]]
assert n_queens_solutions_test1 == [[1, 3, 5, 0, 2, 4], [2, 5, 1, 4, 0, 3],\
                        [3, 0, 4, 1, 5, 2], [4, 2, 0, 5, 3, 1]]
assert n_queens_solutions_test2 == 92
print("test passed!")


AssertionError: 